In [ ]:
### TO DO
## ITEMS:
# Ores (DONE)
# Components (DONE)
#- Grab the standard recipe for each component
# Equipment
## BUILDINGS (DONE)
## RECIPES
#- Obtain alternate recipes from Hard Drive page

In [1]:
import requests
import bs4
base_url = 'https://satisfactory.fandom.com/wiki/{}'
import numpy as np
import pandas as pd
import re

In [6]:
### This is the code to grab the lists of items

##Gets the list of ore names
def get_ore_names():
    ores = requests.get('https://satisfactory.fandom.com/wiki/Category:Ores')
    soup = bs4.BeautifulSoup(ores.text,"lxml")
    oretext = soup.select('div.mw-category-group')
    ore_list = set()

    txt = oretext[0]

    for item in oretext:
        new = item.find_all('a')
        for sub in new:
            ore_list.add(sub['title'])

    return list(ore_list)

In [404]:

def get_comp_dict():
    '''Ouputs a list of components and their corresponding data in DataFrame format'''
    #Gets the list of components and conveniently the tier data as part of a filter 
    # because I'm bad at scraping.
    #Ouputs a dictionary of Component Name:Tier
    #Start by getting the ores witht their tiers.
    item_page = requests.get('https://satisfactory.fandom.com/wiki/Satisfactory_Wiki')
    soup = bs4.BeautifulSoup(item_page.text,"lxml")
    componentvomit = soup.select('.navbox')[14]
    example = componentvomit.find_all('a', title=True)
    comp_list = {}
    comp_dict = {}
    cat = ""
    for item in example:
        if 'Tier' in item['title'] or 'MAM' in item['title']:
            cat = item['title']
            cat.replace('Tier ','')
        else:
            comp_list[item['title']] = cat
    comp_list['Biomass'] = 'Tier 0'
    comp_list.pop('Research',None)
    
    comp_keys = [*comp_list]
    
    ##Extract relevant values from item pages
    tier_list = []
    recipe_list_list = []
    
    for key in comp_keys:
        
        page = requests.get(base_url.format(key))
        soup = bs4.BeautifulSoup(page.text,"lxml")
        
        #Research Tier
        # MAM recipes don't have tiers
        if comp_list[key] == "MAM":
            infobox = soup.select('.portable-infobox')
            a_s = infobox[0].find_all('a')[1]
            try:
                tier = a_s['title']
                tier = tier.replace("Tier ", "")
                #maybe remove this replace function and add it in when the list is called.
            except:
                tier = "Not Implemented"
        else:    
            tier = comp_list[key]
        
        #Stack size and sink value
        aside = soup.select('aside')[0]
        index = 0
        for line in aside.find_all('a'):
            if line['title'] == 'Stack':
                stack_index = index*2
                stack_size = aside.find_all('div')[stack_index].contents[3].text
            elif line['title'] == 'AWESOME Sink':
                sink_index = index*2
                sink_value = aside.find_all('div')[sink_index].contents[3].text
            index += 1
        unlock = aside.find_all('a')[1]['title']
        
        #Alternate Recipe Names
        try:
            craft = soup.select('.wikitable')[0]
            rows = craft.find_all('tr')
        except:
            craft = []

        alt_recipes = []
        for ind in range(8):
            try:
                if "Alternate" in rows[ind+2].find('td').text:
                    alt = rows[ind+2].find('td').text
                    alt.replace("Alternate","")
                    altalt = re.sub(r'Alternate','',alt)
                    altalt = re.sub(r": ", "", altalt)
                    alt_recipes.append(altalt)
            except:
                pass
        
        #Append values to lists
        tier_list.append(tier)
        sink_value_list.append(sink_value)
        recipe_list_list.append(alt_recipes)
    
    comp_dict["Tier"] = tier_list
    comp_dict["Alternate Recipes"] = recipe_list_list
    #Last step - return the names of items and comp_dict in the same order
    return comp_keys, comp_dict
        

In [405]:
comp_keys, comp_dict = get_comp_dict()
ComponentTable = pd.DataFrame(comp_dict, index=comp_keys)
ComponentTable.head()



IndexError: list index out of range

In [123]:
ComponentTable.to_pickle('../Data/Components')

In [289]:
#Grab important values for each ore
#Tier, Sink Value, Mining Speed, No. of nodes 
def get_ore_dict(oredered_list=get_ore_names()):
    ore_dict = {}
    unlock_list = []
    stack_size_list= []
    sink_value_list = []
    mining_list = []
    node_dict = {'Impure':0, 'Normal':0, 'Pure':0}
    impure_list = []
    normal_list = []
    pure_list = []
    
    for ore in oredered_list:
        
        name = requests.get(base_url.format(ore))
        soup = bs4.BeautifulSoup(name.text,"lxml")

        #Tier Acquired, Stack Size, Sink Value
        aside = soup.select('aside')[0]
        index = 0
        for line in aside.find_all('a'):
            if line['title'] == 'Stack':
                stack_index = index*2
                stack_size = aside.find_all('div')[stack_index].contents[3].text
            elif line['title'] == 'AWESOME Sink':
                sink_index = index*2
                sink_value = aside.find_all('div')[sink_index].contents[3].text
            index += 1
        unlock = aside.find_all('a')[1]['title']
        
        # SAM Ore page format was significantly different so I gave up trying to automate this
        # WHEN UPDATING CHECK THIS MANUALLY
        if ore == "SAM Ore":
            stack_size = 50
            sink_value = 'N/A'

        #Mining Speed - Mk.1 - Impure
        for item in soup.select('tbody'):
            if item.find('a')['title'] == 'Miner Mk.1':
                mining = item
                break

        speed = mining.find_all('tr')[1].contents[1].contents[0].text

        #No. of nodes - populates node_dict with node types and values
        for line in soup.select('tbody'):
            if "Numberof nodes" in line.text:
                nodes = line
        node_names = ["Impure","Normal","Pure"]
        follow = False
        key = ''
        for line in nodes.find_all('td'):
            if follow:
                node_dict[key] = line.text
            if any(node in line.text for node in node_names):
                key = line.text
                follow = True
            else:
                follow = False

        #Append new values to list for dictionary
        unlock_list.append(unlock)
        stack_size_list.append(stack_size)
        sink_value_list.append(sink_value)
        mining_list.append(speed)
        impure_list.append(node_dict['Impure'])
        normal_list.append(node_dict['Normal'])
        pure_list.append(node_dict['Pure'])
        
    ore_dict["Unlocks at"] = unlock_list
    ore_dict["Stack Size"] = stack_size_list
    ore_dict["Sink Value"] = sink_value_list
    ore_dict["Mining Speed"] = mining_list
    ore_dict["Impure Nodes"] = impure_list
    ore_dict["Normal Nodes"] = normal_list
    ore_dict["Pure Nodes"] = pure_list
        
    return ore_dict
    
    #Final section will take all previous results and append them all to a DataFrame

In [290]:
oredered_list = get_ore_names()
ore_dict = get_ore_dict(oredered_list=oredered_list)
OreTable = pd.DataFrame(ore_dict, index=oredered_list)
OreTable.head()

,Unlocks at,Stack Size,Sink Value,Mining Speed,Impure Nodes,Normal Nodes,Pure Nodes
Sulfur,Sulfur Research,100,11,30,1,7,3
Copper Ore,Tier 0,100,3,30,9,28,12
Bauxite,Tier 7,100,8,30,5,6,6
Limestone,Tier 0,100,2,30,12,47,27
Coal,Tier 3,100,3,30,6,29,14


In [291]:
OreTable.to_pickle('../Data/ores')

In [397]:
def get_building_values():
### This is the code to grab the list of buildings
    builres = requests.get("https://satisfactory.fandom.com/wiki/Buildings")
    soup = bs4.BeautifulSoup(builres.text,"lxml")
    #This is the relevant information for buildings: name, ingredients.
    btable = soup.select('.wikitable')[0]

    try1 = btable.select('tr')
    building_dict = {}

    #Grabs all 355 building names
    for item in try1:

        tds = item.select('td')

        try:
            bs = tds[1].select('b')
            a_s = tds[1].select('a')

            #Grabs building names
            Building = tds[0].text


            #Grabs ingredient quantities
            blist = []
            for item in bs:
                blist.append(item.text)
            blist

            #Grabs ingredient names
            adict = {}
            for item in a_s:
                adict[item['title']]=""
            #Combines all into a dict of buildings: dict of ingredients to quantities
            
            building_dict[Building] = {'Recipe': dict(zip(adict, blist))}
        except Exception as e:
            pass

    # Grab the dimensions of all buildings
    i = 0
    for item in soup.select('tbody')[2].select('td'):
        try:
            if i == 0 or i%5 == 0:
                key = item.find('a')['title']
                try:
                    superkey = building_dict[key]
                except:
                    superkey = building_dict[key+'FICS⁕MAS']
            elif i%5 == 1:
                superkey['Width'] = item.text
            elif i%5 == 2:
                superkey['Length'] = item.text
            elif i%5 == 3:
                superkey['Height'] = item.text
            elif i%5 == 4:
                superkey['Area'] = item.text
            i += 1
        except:
            pass
    return building_dict


In [398]:
building_dict = get_building_values()
building_table = pd.DataFrame.from_dict(building_dict, orient='index')

In [401]:
building_table.to_pickle('../Data/buildings')

In [304]:
builres = requests.get("https://satisfactory.fandom.com/wiki/Buildings")
soup = bs4.BeautifulSoup(builres.text,"lxml")
btable = soup.select('.wikitable')[0]

In [ ]:
### This is the code to grab the list of recipes
#Because of the way recipes are stored in the wiki, 
#you have to get the basic recipe from each item page
#Then you can get the list of alternate recipes from the hard drive page

##Relevant data to get for each recipe:
#Recipe Name,Product, Building, Ingredients, Rate/min, Prerequisites

In [402]:
alt_recipes_list = ComponentTable["Alternate Recipes"].tolist()
alt_recipes_set = set()
alt_recipes_dict = {}
for item in alt_recipes_list:
    for sub in item:
        alt_recipes_set.add(sub)
        alt_recipes_dict[sub] = "Ingredients"


NameError: name 'ComponentTable' is not defined

In [219]:
alternates = requests.get('https://satisfactory.fandom.com/wiki/Hard_Drive')
soup = bs4.BeautifulSoup(alternates.text,'lxml')
test = soup.select('.wikitable')[1]

In [258]:
test = soup.select('.wikitable')[1]
another = test.find_all('tr')[8]
idict = {}
key = ""
#for item in another:
onemore = another.find_all('a')
key = onemore[0].text
another.find_all('td')
#code is smth like for each item in test.find_all('tr')
#tr[1] is the name
#tr[2] is the product
#tr[3] contains ingredients
#tr[4] is the rate/min
#tr[7] is the prerequisites

[<td style="text-align:left;position:relative; overflow:hidden;padding-top:2.2em;padding-bottom:2.2em;"><a class="mw-redirect" href="/wiki/Nobelisk#Alternate_recipes_analysis" title="Nobelisk">Seismic Nobelisk</a></td>,
 <td><a class="image" href="/wiki/File:Nobelisk.png"><img alt="Nobelisk.png" data-file-height="256" data-file-width="256" decoding="async" height="30" src="https://static.wikia.nocookie.net/satisfactory_gamepedia_en/images/7/76/Nobelisk.png/revision/latest/scale-to-width-down/30?cb=20190430195815" width="30"/></a> <a class="mw-redirect" href="/wiki/Nobelisk" title="Nobelisk">Nobelisk</a></td>,
 <td style="vertical-align:right;"><a class="image" href="/wiki/File:Black_Powder.png"><img alt="Black Powder.png" data-file-height="256" data-file-width="256" decoding="async" height="30" src="https://static.wikia.nocookie.net/satisfactory_gamepedia_en/images/3/3e/Black_Powder.png/revision/latest/scale-to-width-down/30?cb=20190430200121" width="30"/></a> <a href="/wiki/Black_Powd